In [1]:
import pandas as pd
import numpy as np
import json
import requests
import sqlite3
import math
import calendar


# Retrieve data using API

In [2]:
# Retrieve data - API
# response = requests.get('https://covid-api.com/api/regions')
# regions = response.json()

# response = requests.get('https://covid-api.com/api/provinces')
# provinces = response.json()

# response = requests.get('https://covid-api.com/api/reports/total')
# total = response.json()

response = requests.get('https://covid-api.com/api/reports')

reports = response.json()

In [3]:
reports

{'data': [{'date': '2023-03-09',
   'confirmed': 209451,
   'deaths': 7896,
   'recovered': 0,
   'confirmed_diff': 0,
   'deaths_diff': 0,
   'recovered_diff': 0,
   'last_update': '2023-03-10 04:21:03',
   'active': 201555,
   'active_diff': 0,
   'fatality_rate': 0.0377,
   'region': {'iso': 'AFG',
    'name': 'Afghanistan',
    'province': '',
    'lat': '33.9391',
    'long': '67.7100',
    'cities': []}},
  {'date': '2023-03-09',
   'confirmed': 334457,
   'deaths': 3598,
   'recovered': 0,
   'confirmed_diff': 14,
   'deaths_diff': 0,
   'recovered_diff': 0,
   'last_update': '2023-03-10 04:21:03',
   'active': 330859,
   'active_diff': 14,
   'fatality_rate': 0.0108,
   'region': {'iso': 'ALB',
    'name': 'Albania',
    'province': '',
    'lat': '41.1533',
    'long': '20.1683',
    'cities': []}},
  {'date': '2023-03-09',
   'confirmed': 271496,
   'deaths': 6881,
   'recovered': 0,
   'confirmed_diff': 2,
   'deaths_diff': 0,
   'recovered_diff': 0,
   'last_update': '2023-

In [4]:
# create df for reports
reports_json = reports['data']
df = pd.DataFrame.from_dict(reports_json) 
df

,date,confirmed,deaths,recovered,confirmed_diff,deaths_diff,recovered_diff,last_update,active,active_diff,fatality_rate,region
0,2023-03-09,209451,7896,0,0,0,0,2023-03-10 04:21:03,201555,0,0.0377,"{'iso': 'AFG', 'name': 'Afghanistan', 'provinc..."
1,2023-03-09,334457,3598,0,14,0,0,2023-03-10 04:21:03,330859,14,0.0108,"{'iso': 'ALB', 'name': 'Albania', 'province': ..."
2,2023-03-09,271496,6881,0,2,0,0,2023-03-10 04:21:03,264615,2,0.0253,"{'iso': 'DZA', 'name': 'Algeria', 'province': ..."
3,2023-03-09,47890,165,0,0,0,0,2023-03-10 04:21:03,47725,0,0.0034,"{'iso': 'AND', 'name': 'Andorra', 'province': ..."
4,2023-03-09,105288,1933,0,0,0,0,2023-03-10 04:21:03,103355,0,0.0184,"{'iso': 'AGO', 'name': 'Angola', 'province': '..."
...,...,...,...,...,...,...,...,...,...,...,...,...
786,2023-03-09,11526994,43186,0,0,0,0,2023-03-10 04:21:03,11483808,0,0.0037,"{'iso': 'VNM', 'name': 'Vietnam', 'province': ..."
787,2023-03-09,703228,5708,0,0,0,0,2023-03-10 04:21:03,697520,0,0.0081,"{'iso': 'PSE', 'name': 'West Bank and Gaza', '..."
788,2023-03-09,11945,2159,0,0,0,0,2023-03-10 04:21:03,9786,0,0.1807,"{'iso': 'YEM', 'name': 'Yemen', 'province': ''..."
789,2023-03-09,343135,4057,0,0,0,0,2023-03-10 04:21:03,339078,0,0.0118,"{'iso': 'ZMB', 'name': 'Zambia', 'province': '..."


# Data Preprocessing

In [5]:
# split df['region'] data to columns
region_info =  df['region'].apply(pd.Series)
df = pd.concat([df.drop(['region'], axis=1),region_info], axis=1)


In [6]:
df

,date,confirmed,deaths,recovered,confirmed_diff,deaths_diff,recovered_diff,last_update,active,active_diff,fatality_rate,iso,name,province,lat,long,cities
0,2023-03-09,209451,7896,0,0,0,0,2023-03-10 04:21:03,201555,0,0.0377,AFG,Afghanistan,,33.9391,67.7100,[]
1,2023-03-09,334457,3598,0,14,0,0,2023-03-10 04:21:03,330859,14,0.0108,ALB,Albania,,41.1533,20.1683,[]
2,2023-03-09,271496,6881,0,2,0,0,2023-03-10 04:21:03,264615,2,0.0253,DZA,Algeria,,28.0339,1.6596,[]
3,2023-03-09,47890,165,0,0,0,0,2023-03-10 04:21:03,47725,0,0.0034,AND,Andorra,,42.5063,1.5218,[]
4,2023-03-09,105288,1933,0,0,0,0,2023-03-10 04:21:03,103355,0,0.0184,AGO,Angola,,-11.2027,17.8739,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,2023-03-09,11526994,43186,0,0,0,0,2023-03-10 04:21:03,11483808,0,0.0037,VNM,Vietnam,,14.0583,108.2772,[]
787,2023-03-09,703228,5708,0,0,0,0,2023-03-10 04:21:03,697520,0,0.0081,PSE,West Bank and Gaza,,31.9522,35.2332,[]
788,2023-03-09,11945,2159,0,0,0,0,2023-03-10 04:21:03,9786,0,0.1807,YEM,Yemen,,15.552726999999999,48.516388,[]
789,2023-03-09,343135,4057,0,0,0,0,2023-03-10 04:21:03,339078,0,0.0118,ZMB,Zambia,,-13.1339,27.8493,[]


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 791 entries, 0 to 790
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            791 non-null    object 
 1   confirmed       791 non-null    int64  
 2   deaths          791 non-null    int64  
 3   recovered       791 non-null    int64  
 4   confirmed_diff  791 non-null    int64  
 5   deaths_diff     791 non-null    int64  
 6   recovered_diff  791 non-null    int64  
 7   last_update     791 non-null    object 
 8   active          791 non-null    int64  
 9   active_diff     791 non-null    int64  
 10  fatality_rate   791 non-null    float64
 11  iso             791 non-null    object 
 12  name            791 non-null    object 
 13  province        791 non-null    object 
 14  lat             775 non-null    object 
 15  long            775 non-null    object 
 16  cities          791 non-null    object 
dtypes: float64(1), int64(8), object(8)


In [8]:
df.describe()

,confirmed,deaths,recovered,confirmed_diff,deaths_diff,recovered_diff,active,active_diff,fatality_rate
count,7.910000e+02,791.000000,791.0,791.000000,791.000000,791.0,7.910000e+02,791.000000,791.000000
mean,8.553031e+05,8700.046776,0.0,245.386852,2.343869,0.0,8.466031e+05,243.042984,0.015246
std,2.397051e+06,19993.091547,0.0,1412.119417,18.059342,0.0,2.382865e+06,1396.168046,0.025190
min,0.000000e+00,0.000000,0.0,0.000000,-1.000000,0.0,-3.380000e+02,-13.000000,0.000000
25%,6.179550e+04,534.500000,0.0,0.000000,0.000000,0.0,5.898450e+04,0.000000,0.004300
50%,2.111580e+05,2548.000000,0.0,0.000000,0.000000,0.0,2.075050e+05,0.000000,0.010300
75%,6.732080e+05,8297.000000,0.0,89.000000,0.000000,0.0,6.673000e+05,88.500000,0.019100
max,3.861851e+07,186138.000000,0.0,26285.000000,431.000000,0.0,3.845700e+07,25854.000000,0.533600


In [9]:
# Check how 'confirmed' is calculated

check_confirmed = df['deaths'] + df['active'] + df['recovered']
df['confirmed'].equals(check_confirmed) 

True

In [10]:
# Check how 'fatality_rate' is calculated

def round_half_up(n, decimals=0):
    multiplier = 10**decimals
    return math.floor(n * multiplier + 0.5) / multiplier

check_fatality_rate = (df['deaths'] / df['confirmed'])
check_fatality_rate = check_fatality_rate.replace(math.inf, 0)
check_fatality_rate.fillna(0, inplace=True)
check_fatality_rate = check_fatality_rate.apply(lambda x: round_half_up(x, 4))

set(np.isclose(df['fatality_rate'], check_fatality_rate))

{True}

In [11]:
# Check how many countries and how many entries per country
df['name'].value_counts()

name
Russia      83
US          59
Japan       49
India       37
Colombia    34
            ..
Guyana       1
Haiti        1
Holy See     1
Honduras     1
Zimbabwe     1
Name: count, Length: 196, dtype: int64

In [12]:
df['name'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France',
       'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Grenada', 'Guatemala', 'Guin

In [13]:
# Below is to check if country has empty and non empty provinces

check_empty_province = df[df['province'] == '']
list_of_country_with_empty_province = list(check_empty_province['name'])

df_filter_country = df[df['name'].isin(list_of_country_with_empty_province)]
check_no_of_province =  df_filter_country[['name','province']].groupby(['name']).count().reset_index()
check_no_of_province[check_no_of_province['province'] >1]

,name,province
40,Denmark,3
55,France,12
112,New Zealand,3


In [14]:
df[df['name'] == 'Denmark']

,date,confirmed,deaths,recovered,confirmed_diff,deaths_diff,recovered_diff,last_update,active,active_diff,fatality_rate,iso,name,province,lat,long,cities
188,2023-03-09,34658,28,0,0,0,0,2023-03-10 04:21:03,34630,0,0.0008,DNK,Denmark,Faroe Islands,61.8926,-6.9118,[]
189,2023-03-09,11971,21,0,0,0,0,2023-03-10 04:21:03,11950,0,0.0018,DNK,Denmark,Greenland,71.7069,-42.6043,[]
190,2023-03-09,3404407,8296,0,139,4,0,2023-03-10 04:21:03,3396111,135,0.0024,DNK,Denmark,,56.2639,9.5018,[]


In [15]:
# check how many date 
len(df['date'].unique())

1

In [16]:
# check how many last_update
df['last_update'].unique()

array(['2023-03-10 04:21:03', '2020-12-21 13:27:30',
       '2022-11-22 23:21:06', '2023-01-30 23:20:55',
       '2020-08-04 02:27:56'], dtype=object)

In [17]:
# Check that name + province pair has no duplicates
name_province =  df[['name','province','date']].groupby(['name','province']).count().reset_index()
name_province[name_province['date'] >1]

,name,province,date


In [18]:
# count no of rows with empty list in cities column
len(df[df["cities"].str.len() == 0])

740

## Notes from analysing dataset


# Retrieve monthly data from Jan 2022 to Feb 2023

In [19]:
# get monthly data from 2020 to 2023
year = list(range(2020,2024))
month = [str(i).zfill(2) for i in list(range(1,13))]

monthly_raw = pd.DataFrame(columns=['date', 'confirmed', 'deaths', 'recovered', 'confirmed_diff',
       'deaths_diff', 'recovered_diff', 'last_update', 'active', 'active_diff',
       'fatality_rate', 'region'])

for i in year:
    for j in month:
        response = requests.get(f'https://covid-api.com/api/reports?date={i}-{j}-{calendar.monthrange(i, int(j))[1]}')
        report_raw = response.json()
        report_json = report_raw['data']
        month_data = pd.DataFrame.from_dict(report_json) 
        monthly_raw = pd.concat([monthly_raw,month_data], ignore_index=True)
        

In [20]:
monthly_raw['date'].unique()

array(['2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30',
       '2020-05-31', '2020-06-30', '2020-07-31', '2020-08-31',
       '2020-09-30', '2020-10-31', '2020-11-30', '2020-12-31',
       '2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30',
       '2021-05-31', '2021-06-30', '2021-07-31', '2021-08-31',
       '2021-09-30', '2021-10-31', '2021-11-30', '2021-12-31',
       '2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30',
       '2022-05-31', '2022-06-30', '2022-07-31', '2022-08-31',
       '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-31',
       '2023-01-31', '2023-02-28'], dtype=object)

In [21]:
monthly_raw

,date,confirmed,deaths,recovered,confirmed_diff,deaths_diff,recovered_diff,last_update,active,active_diff,fatality_rate,region
0,2020-01-31,5806,204,141,903,42,51,2020-01-31 23:59:00,5461,810,0.0351,"{'iso': 'CHN', 'name': 'China', 'province': 'H..."
1,2020-01-31,538,0,14,110,0,10,2020-01-31 23:59:00,524,100,0.0000,"{'iso': 'CHN', 'name': 'China', 'province': 'Z..."
2,2020-01-31,436,0,11,82,0,1,2020-01-31 23:59:00,425,81,0.0000,"{'iso': 'CHN', 'name': 'China', 'province': 'G..."
3,2020-01-31,352,2,3,74,0,1,2020-01-31 23:59:00,347,73,0.0057,"{'iso': 'CHN', 'name': 'China', 'province': 'H..."
4,2020-01-31,332,0,2,55,0,0,2020-01-31 23:59:00,330,55,0.0000,"{'iso': 'CHN', 'name': 'China', 'province': 'H..."
...,...,...,...,...,...,...,...,...,...,...,...,...
26868,2023-02-28,11526917,43186,0,12,0,0,2023-03-01 04:21:02,11483731,12,0.0037,"{'iso': 'VNM', 'name': 'Vietnam', 'province': ..."
26869,2023-02-28,703228,5708,0,0,0,0,2023-03-01 04:21:02,697520,0,0.0081,"{'iso': 'PSE', 'name': 'West Bank and Gaza', '..."
26870,2023-02-28,11945,2159,0,0,0,0,2023-03-01 04:21:02,9786,0,0.1807,"{'iso': 'YEM', 'name': 'Yemen', 'province': ''..."
26871,2023-02-28,343012,4057,0,181,2,0,2023-03-01 04:21:02,338955,179,0.0118,"{'iso': 'ZMB', 'name': 'Zambia', 'province': '..."


In [22]:
# split df['region'] data to columns
monthly_region_info =  monthly_raw['region'].apply(pd.Series)
monthly_df = pd.concat([monthly_raw.drop(['region'], axis=1),monthly_region_info], axis=1)
monthly_df

,date,confirmed,deaths,recovered,confirmed_diff,deaths_diff,recovered_diff,last_update,active,active_diff,fatality_rate,iso,name,province,lat,long,cities
0,2020-01-31,5806,204,141,903,42,51,2020-01-31 23:59:00,5461,810,0.0351,CHN,China,Hubei,30.9756,112.2707,[]
1,2020-01-31,538,0,14,110,0,10,2020-01-31 23:59:00,524,100,0.0000,CHN,China,Zhejiang,29.1832,120.0934,[]
2,2020-01-31,436,0,11,82,0,1,2020-01-31 23:59:00,425,81,0.0000,CHN,China,Guangdong,23.3417,113.4244,[]
3,2020-01-31,352,2,3,74,0,1,2020-01-31 23:59:00,347,73,0.0057,CHN,China,Henan,33.8820,113.6140,[]
4,2020-01-31,332,0,2,55,0,0,2020-01-31 23:59:00,330,55,0.0000,CHN,China,Hunan,27.6104,111.7088,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26868,2023-02-28,11526917,43186,0,12,0,0,2023-03-01 04:21:02,11483731,12,0.0037,VNM,Vietnam,,14.0583,108.2772,[]
26869,2023-02-28,703228,5708,0,0,0,0,2023-03-01 04:21:02,697520,0,0.0081,PSE,West Bank and Gaza,,31.9522,35.2332,[]
26870,2023-02-28,11945,2159,0,0,0,0,2023-03-01 04:21:02,9786,0,0.1807,YEM,Yemen,,15.552726999999999,48.516388,[]
26871,2023-02-28,343012,4057,0,181,2,0,2023-03-01 04:21:02,338955,179,0.0118,ZMB,Zambia,,-13.1339,27.8493,[]


In [23]:
# selecting columns that are useful for future analysis
# Group the columns to get aggregate data 

df_cols = monthly_df[['date', 'confirmed', 'deaths', 'iso', 'name']]

df_cleaned = df_cols.groupby(['date', 'iso', 'name']).agg({"confirmed":"sum", "deaths":"sum"})
df_cleaned.reset_index(inplace= True)
df_cleaned

,date,iso,name,confirmed,deaths
0,2020-01-31,ARE,United Arab Emirates,4,0
1,2020-01-31,AUS,Australia,9,0
2,2020-01-31,CAN,Canada,3,0
3,2020-01-31,CHN,China,9802,213
4,2020-01-31,DEU,Germany,5,0
...,...,...,...,...,...
6996,2023-02-28,WSM,Samoa,16607,29
6997,2023-02-28,YEM,Yemen,11945,2159
6998,2023-02-28,ZAF,South Africa,4063573,102595
6999,2023-02-28,ZMB,Zambia,343012,4057


In [24]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7001 entries, 0 to 7000
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       7001 non-null   object
 1   iso        7001 non-null   object
 2   name       7001 non-null   object
 3   confirmed  7001 non-null   object
 4   deaths     7001 non-null   object
dtypes: object(5)
memory usage: 273.6+ KB


# Save preprocessed data to db

In [25]:
# Establish a connection to the SQLite database:
conn = sqlite3.connect('iod_project_1_data.db')

# Save dataframe (orders) as a table in sqlite
df_cleaned.to_sql('preprocessed_data', con = conn, if_exists = 'replace', index = False)

# Close the Connection:
# Once you have finished working with the database, it is important to close the connection:
conn.close()